In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print


Mounted at /content/drive


In [6]:
import numpy as np
import pandas as pd

In [7]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

pd.options.display.max_columns = None

In [8]:
import os
findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '10b', tickers, False)

In [10]:
addAlloc(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
17,DOCN,1.18,0.97,597.50,629.74,39.60,46.47,0.17,0.93,0.81,811.99,814.50,41.89,0.06,0.70,0.77,422.27,423.95,38.81,-0.02,9.523529
3,AMBA,2.47,0.96,1087.56,1092.33,63.62,77.34,0.22,1.92,0.71,738.60,970.64,67.29,0.06,1.25,0.74,839.30,1142.15,59.63,-0.06,9.418182
77,TRIP,0.53,0.97,213.28,298.63,13.87,15.08,0.09,0.45,0.71,161.04,310.69,15.35,0.11,0.33,0.81,232.39,329.19,13.58,-0.02,9.366667
36,INMD,0.93,0.95,401.28,550.16,19.02,23.23,0.22,0.64,0.63,111.55,326.51,21.45,0.13,0.53,0.83,477.51,583.28,19.84,0.04,9.272727
63,PUBM,0.58,0.95,203.57,448.06,16.18,18.35,0.13,0.43,0.76,269.77,363.40,17.35,0.07,0.36,0.77,242.43,342.42,15.18,-0.06,9.115385
84,WOLF,2.68,0.92,626.93,1174.76,8.44,16.19,0.92,1.72,0.63,562.02,604.51,17.05,1.02,1.45,0.75,733.06,941.14,10.74,0.27,9.113043
85,VEEV,3.23,0.97,1512.52,1660.58,216.58,226.50,0.05,2.10,0.72,1019.42,1173.00,242.01,0.12,2.03,0.78,1410.20,1805.68,203.69,-0.06,9.100000
14,DASH,2.43,0.95,1242.40,983.86,177.24,198.74,0.12,1.66,0.76,1127.33,908.19,192.27,0.08,1.69,0.82,1445.30,1396.76,162.83,-0.08,9.083333
76,TOST,0.65,0.95,179.84,231.25,42.74,46.85,0.10,0.55,0.75,208.61,159.79,42.46,-0.01,0.32,0.77,199.15,267.65,37.24,-0.13,9.000000
88,ZS,4.66,0.94,2159.72,1926.89,210.96,242.24,0.15,2.98,0.70,1529.33,1419.63,205.25,-0.03,2.99,0.81,2655.69,2784.37,184.36,-0.13,9.000000


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '11b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [12]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
32,VUG,1.90,0.99,1654.78,762.94,404.03,395.44,-0.02,1.46,0.74,1249.86,676.19,414.23,0.03,1.40,0.63,1036.22,430.79,387.95,-0.04,9.400000
23,ONLN,0.67,0.96,182.99,400.53,44.93,48.97,0.09,0.53,0.61,40.05,143.01,47.15,0.05,0.62,0.74,175.88,394.42,43.70,-0.03,9.155556
11,IPO,0.61,0.96,249.98,348.81,46.12,50.39,0.09,0.47,0.67,102.87,108.78,46.53,0.01,0.40,0.74,242.61,408.04,42.92,-0.07,9.155556
45,XLY,1.31,0.95,829.73,596.34,218.10,186.63,-0.14,0.81,0.74,640.50,507.57,215.14,-0.01,0.85,0.71,768.79,786.79,192.34,-0.12,9.142857
43,XLU,0.43,0.97,296.28,231.99,81.45,77.51,-0.05,0.26,0.75,269.96,140.42,80.52,-0.01,0.25,0.68,252.44,129.12,75.53,-0.07,9.100000
9,ICLN,0.23,0.94,84.80,141.16,12.12,14.16,0.17,0.14,0.63,51.46,82.82,14.18,0.17,0.15,0.78,81.96,146.98,11.51,-0.05,9.047059
48,XSW,1.39,0.94,708.10,576.87,194.96,167.81,-0.14,0.75,0.72,535.00,473.47,193.67,-0.01,0.84,0.72,647.14,794.84,165.94,-0.15,8.971429
36,XLC,0.65,0.95,319.51,203.52,96.88,88.96,-0.08,0.38,0.82,293.31,214.83,99.74,0.03,0.36,0.66,214.22,182.18,79.90,-0.18,8.875000
4,DAPP,0.41,0.95,157.20,176.44,18.47,16.97,-0.08,0.26,0.67,99.18,118.68,19.38,0.05,0.21,0.66,82.35,81.45,13.43,-0.27,8.875000
15,IWM,1.45,0.94,721.22,760.05,238.77,216.94,-0.09,0.71,0.76,1005.29,759.06,235.81,-0.01,0.77,0.79,921.38,1100.68,218.26,-0.09,8.733333


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO',  'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON',
            'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'TELL',
            'VTEX', 'VSAT', 'YEXT', 'WKME']
df3 = run_model('vols', '10b', tickers3, True)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
54,REAL,0.45,0.93,60.74,187.53,2.93,3.54,0.21,0.62,0.50,5.22,111.40,3.47,0.18,0.26,0.78,69.31,178.52,2.45,-0.16,5.137255
36,LAZR,0.48,0.90,83.85,216.78,0.81,1.14,0.40,0.97,0.43,-20.89,119.19,1.02,0.25,0.37,0.78,77.03,195.97,0.83,0.02,5.044118
14,BLDE,0.23,0.92,60.47,116.59,3.54,4.28,0.21,0.46,0.47,-26.58,25.73,4.14,0.17,0.20,0.69,46.95,72.67,3.50,-0.01,5.030812
68,STEM,0.66,0.89,191.95,258.65,0.54,0.80,0.48,0.92,0.50,23.88,142.95,0.82,0.51,0.42,0.75,85.92,231.57,0.62,0.15,5.006127
17,CHGG,1.33,0.86,179.15,510.23,1.60,3.75,1.34,2.15,0.51,-99.48,188.45,2.24,0.40,1.17,0.65,132.21,546.24,2.07,0.29,4.954346
45,OMQS,0.21,0.86,59.04,78.39,0.17,0.30,0.76,0.50,0.49,54.55,69.58,0.26,0.55,0.17,0.68,40.62,69.33,0.19,0.11,4.874613
73,VTEX,0.27,0.90,33.72,91.43,6.64,7.96,0.20,0.51,0.58,-6.10,18.09,7.55,0.14,0.27,0.67,31.55,75.42,6.28,-0.05,4.794118
60,SABR,0.23,0.91,76.53,85.03,3.99,4.62,0.16,0.51,0.49,-11.86,34.13,4.63,0.16,0.20,0.72,47.09,100.00,3.57,-0.10,4.790441
33,INVZ,0.17,0.91,38.97,79.10,0.69,0.80,0.16,0.40,0.50,-8.79,34.15,0.85,0.24,0.18,0.79,41.90,102.08,0.66,-0.04,4.790441
15,BLNK,0.74,0.88,127.61,353.67,2.20,2.69,0.22,1.87,0.46,-33.69,107.82,2.53,0.15,0.55,0.71,135.88,295.91,2.07,-0.06,4.631016


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '9b', tickers4, False)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
3,AMD,2.37,0.96,1478.70,1027.96,141.86,155.59,0.10,2.04,0.74,1301.42,941.10,174.95,0.23,1.68,0.68,1059.27,617.30,127.29,-0.10,6.457143
19,FDX,3.66,0.94,1356.44,1514.52,274.35,238.00,-0.13,2.24,0.72,1139.05,1150.88,276.81,0.01,2.15,0.72,1471.59,897.15,213.04,-0.22,6.252747
2,ADBE,8.09,0.94,3287.07,4111.16,482.80,531.27,0.10,5.13,0.70,3147.91,2411.67,519.98,0.08,5.10,0.73,2825.80,3587.43,467.02,-0.03,6.114286
4,AMZN,2.83,0.93,1106.90,653.63,197.93,172.84,-0.13,1.65,0.71,613.76,575.20,190.40,-0.04,1.73,0.64,613.92,590.35,174.18,-0.12,6.104396
6,BA,4.59,0.91,1226.23,1510.93,154.59,179.42,0.16,3.19,0.67,745.16,1082.58,167.83,0.09,2.74,0.67,777.30,1160.93,134.09,-0.13,5.937500
25,META,11.54,0.91,2827.23,1117.40,567.16,510.87,-0.10,8.16,0.72,1891.33,898.32,568.10,0.00,7.70,0.64,2139.50,226.67,465.37,-0.18,5.600000
33,QCOM,2.46,0.96,1147.41,894.06,165.27,170.58,0.03,1.60,0.74,877.31,896.98,175.17,0.06,1.41,0.66,755.81,534.60,152.91,-0.07,5.523810
23,ISRG,8.82,0.89,2477.73,1396.08,506.34,442.55,-0.13,6.61,0.68,1620.88,869.34,508.78,0.00,4.11,0.71,1949.31,1038.52,452.70,-0.11,5.510989
34,SBUX,1.28,0.92,454.99,485.31,98.87,91.91,-0.07,0.78,0.66,404.12,367.94,95.47,-0.03,0.83,0.74,357.38,320.52,93.14,-0.06,5.428571
30,NOW,13.37,0.89,3981.04,2631.34,943.41,854.55,-0.09,12.94,0.65,3448.51,1221.70,959.37,0.02,7.12,0.70,3484.25,3869.40,855.17,-0.09,5.134921


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '11a', tickers5, True)

In [ ]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
8,COTY,0.23,0.94,78.58,55.57,7.48,8.47,0.13,0.13,0.68,74.90,50.57,9.11,0.22,0.12,0.68,51.17,51.72,7.20,-0.04,5.791209
56,STLA,0.33,0.90,112.43,122.22,13.70,16.97,0.24,0.20,0.74,76.70,69.64,15.77,0.15,0.15,0.77,107.81,109.92,12.28,-0.10,5.595238
45,OTGLY,0.37,0.88,72.06,100.36,9.86,11.93,0.21,0.28,0.69,33.30,74.23,11.91,0.21,0.22,0.68,45.26,96.42,10.08,0.02,5.142857
31,HMC,0.35,0.97,125.67,137.00,30.29,31.20,0.03,0.19,0.69,113.12,97.66,32.53,0.07,0.19,0.75,127.85,113.37,29.99,-0.01,4.928571
64,VALE,0.22,0.91,101.29,81.12,10.60,11.76,0.11,0.11,0.77,108.33,105.59,12.36,0.17,0.13,0.64,50.74,79.98,10.76,0.01,4.863636
3,BAK,0.34,0.92,139.18,147.51,5.89,6.43,0.09,0.24,0.71,143.37,123.69,8.70,0.48,0.26,0.74,95.86,139.99,5.85,-0.01,4.793651
48,PROSF,1.96,0.89,390.23,708.12,41.67,47.67,0.14,1.49,0.66,193.88,446.72,48.08,0.15,0.92,0.81,387.75,552.45,40.65,-0.02,4.785714
26,GLOB,4.97,0.92,1904.99,1576.37,210.12,227.89,0.08,2.48,0.71,1712.80,2032.20,232.79,0.11,2.37,0.78,1888.28,2016.57,204.87,-0.02,4.571429
17,EQNR,0.50,0.89,162.46,121.96,23.28,26.01,0.12,0.24,0.74,162.58,106.45,28.02,0.20,0.23,0.67,147.77,85.65,23.39,0.00,4.523810
2,ADYEY,0.54,0.91,149.18,199.50,15.23,16.54,0.09,0.32,0.74,187.05,199.35,17.88,0.17,0.29,0.74,136.10,174.69,14.41,-0.05,4.500000


In [ ]:
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '8a', tickers6, False)

In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
21,TUR,0.55,0.87,177.69,143.53,34.52,29.60,-0.14,0.52,0.55,73.28,88.15,34.73,0.01,0.48,0.63,109.61,101.32,29.21,-0.15,7.771429
4,EPHE,0.42,0.87,98.64,100.72,26.73,30.12,0.13,0.29,0.73,60.80,66.29,29.52,0.10,0.45,0.64,57.04,57.07,27.04,0.01,7.700000
19,KEMQ,0.68,0.84,52.00,147.51,17.91,21.48,0.20,0.41,0.65,8.03,103.54,22.82,0.27,0.39,0.78,66.26,143.15,16.23,-0.09,7.600000
14,EWY,1.28,0.84,176.35,280.28,59.20,65.63,0.11,0.72,0.71,107.00,215.24,62.30,0.05,0.90,0.72,196.48,298.05,57.48,-0.03,6.945455
22,VNM,0.20,0.89,62.85,64.76,11.92,12.47,0.05,0.15,0.68,30.67,49.65,12.61,0.06,0.13,0.76,53.64,77.04,12.03,0.01,6.700000
3,ECH,0.71,0.86,113.02,119.82,25.24,26.31,0.04,0.55,0.71,93.81,85.20,25.75,0.02,0.63,0.65,100.49,78.17,23.57,-0.07,5.100000
17,ILF,0.72,0.78,74.50,63.94,24.73,23.34,-0.06,0.45,0.71,83.89,69.70,24.60,-0.01,0.64,0.75,81.28,86.96,22.70,-0.08,4.133333
16,IDX,0.57,0.74,27.43,45.94,16.46,17.62,0.07,0.38,0.56,12.18,15.12,17.98,0.09,0.30,0.67,25.46,36.84,16.02,-0.03,3.685714
12,EWT,1.08,0.80,184.21,107.52,55.99,53.88,-0.04,0.84,0.68,130.91,84.93,55.81,-0.00,0.92,0.70,135.64,101.91,47.97,-0.14,3.000000
13,EWW,1.45,0.80,221.45,160.97,52.19,50.69,-0.03,1.23,0.67,179.61,99.57,58.45,0.12,1.29,0.66,140.82,78.13,47.46,-0.09,1.333333


In [ ]:
findata.EPOCHS=200

tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']
df7 = run_model('val', '11a', tickers7, True)

In [ ]:
addAlloc(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
18,CC,0.54,0.94,228.76,243.02,19.79,22.42,0.13,0.33,0.79,303.40,254.90,21.92,0.11,0.30,0.64,217.63,198.73,17.68,-0.11,8.938462
86,SWK,1.88,0.93,651.48,577.02,90.17,105.93,0.17,1.25,0.71,484.22,680.77,105.45,0.17,1.22,0.76,508.08,897.87,88.74,-0.02,8.888235
2,ALB,4.25,0.95,2157.52,2119.46,100.84,108.97,0.08,3.57,0.70,1989.41,1673.02,114.38,0.13,2.96,0.66,1166.18,1484.67,85.39,-0.15,8.875000
89,TGT,2.77,0.95,1067.30,1158.84,149.78,157.93,0.05,1.76,0.67,1024.35,931.12,160.29,0.07,1.68,0.64,784.64,632.93,137.39,-0.08,8.500000
8,ARE,2.04,0.93,828.72,898.09,114.04,122.86,0.08,1.17,0.73,718.88,754.38,124.65,0.09,1.11,0.75,696.30,957.50,108.35,-0.05,8.425000
13,BGFV,0.43,0.88,164.57,126.56,1.71,2.13,0.25,0.35,0.61,124.42,180.79,2.37,0.39,0.23,0.70,98.32,136.23,1.71,0.00,8.320000
9,ASC,0.29,0.87,106.26,73.28,12.15,16.14,0.33,0.19,0.72,108.43,62.26,15.02,0.24,0.17,0.68,86.67,47.92,10.92,-0.10,8.306061
65,OLN,0.91,0.91,378.32,393.74,44.29,48.98,0.11,0.57,0.78,381.88,394.49,47.17,0.07,0.56,0.71,396.90,321.64,41.55,-0.06,8.281818
56,KHC,0.35,0.96,162.10,157.36,33.00,34.02,0.03,0.18,0.81,186.24,215.65,34.47,0.04,0.16,0.74,136.62,141.25,32.67,-0.01,8.266667
50,IFF,1.37,0.90,534.23,601.72,92.28,102.78,0.11,1.03,0.74,448.62,634.93,112.65,0.22,0.81,0.63,426.95,501.90,95.93,0.04,8.090909


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '10a', tickers8, False)

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
14,WEAT,0.13,0.91,43.86,43.78,4.84,5.54,0.14,0.10,0.70,33.42,41.22,5.55,0.15,0.10,0.74,34.40,51.33,4.91,0.01,8.457143
1,FXE,0.58,0.89,121.44,136.05,97.18,102.64,0.06,0.35,0.67,79.29,113.40,100.64,0.04,0.37,0.73,95.39,135.14,97.42,0.00,7.066667
3,FXY,0.63,0.87,85.41,190.04,59.83,63.87,0.07,0.36,0.59,37.04,124.80,63.72,0.06,0.46,0.70,29.17,150.64,58.62,-0.02,6.842857
6,PDBC,0.23,0.86,45.72,47.31,13.18,14.02,0.06,0.15,0.74,43.57,40.11,13.88,0.05,0.19,0.87,57.51,68.27,13.03,-0.01,6.266667
2,FXF,0.48,0.87,162.07,140.06,100.05,101.22,0.01,0.38,0.67,129.01,121.64,102.34,0.02,0.38,0.73,144.70,117.08,101.36,0.01,0.000000
4,GLD,2.50,0.86,699.72,282.42,236.59,235.21,-0.01,2.23,0.67,420.57,1.35,241.82,0.02,1.74,0.75,465.74,234.29,226.53,-0.04,0.000000
0,FXB,0.67,0.91,228.59,213.47,121.20,121.38,0.00,0.41,0.72,140.83,144.43,126.08,0.04,0.52,0.66,102.75,150.01,121.38,0.00,0.000000
5,ISHG,0.47,0.90,82.36,141.34,69.98,70.73,0.01,0.26,0.73,64.18,108.61,72.00,0.03,0.26,0.78,73.68,131.27,70.82,0.01,0.000000
7,SLV,0.79,0.78,71.29,88.83,27.57,27.29,-0.01,0.52,0.68,97.27,90.78,28.44,0.03,0.46,0.78,101.40,102.54,27.13,-0.02,0.000000
9,TIP,0.43,0.89,113.90,145.05,107.45,108.83,0.01,0.27,0.74,89.71,105.25,109.68,0.02,0.26,0.61,86.62,71.36,107.40,-0.00,0.000000


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, True)

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
16,XRP-USD,0.01,0.94,5.30,6.00,1.19,0.67,-0.43,0.01,0.65,5.35,5.50,0.89,-0.25,0.01,0.83,6.78,6.24,0.65,-0.45,6.574751
4,DOGE-USD,0.00,0.87,2.01,1.05,0.37,0.20,-0.46,0.00,0.64,1.92,1.49,0.29,-0.21,0.00,0.70,1.05,0.20,0.17,-0.53,5.931677
7,FIL-USD,0.31,0.91,63.61,150.08,4.52,5.15,0.14,0.24,0.69,68.01,122.97,5.34,0.18,0.17,0.78,67.79,142.02,3.70,-0.18,5.857143
3,BCH-USD,11.24,0.90,2656.19,3076.23,455.59,379.88,-0.17,7.67,0.67,3269.48,3242.29,512.14,0.12,4.30,0.73,2692.74,1870.97,390.79,-0.14,5.840336
5,DOT-USD,0.32,0.87,58.35,114.71,5.50,6.89,0.25,0.19,0.71,67.99,117.99,6.46,0.18,0.13,0.80,71.14,117.76,4.62,-0.16,5.694286
0,ADA-USD,0.02,0.87,4.23,7.63,0.77,0.60,-0.22,0.01,0.75,5.64,6.58,0.80,0.04,0.01,0.80,5.64,7.41,0.48,-0.37,5.623377
2,BTC-USD,1300.11,0.86,411796.11,249279.03,90959.58,72406.42,-0.20,985.00,0.70,183486.37,205371.61,91783.20,0.01,617.04,0.64,398837.07,219000.88,72879.55,-0.20,5.442857
13,SOL-USD,4.53,0.86,1370.15,864.64,217.06,181.88,-0.16,3.14,0.58,816.40,885.62,226.48,0.04,1.89,0.71,934.07,952.11,179.66,-0.17,5.267857
10,LTC-USD,2.08,0.91,606.77,783.74,92.24,86.12,-0.07,1.09,0.75,808.40,838.78,101.84,0.10,0.88,0.76,873.32,952.84,71.57,-0.22,5.214286
6,ETH-USD,67.88,0.86,25489.45,24124.80,3166.20,2694.01,-0.15,43.87,0.65,10922.14,12587.72,3089.31,-0.02,22.93,0.78,21850.25,19146.17,2862.80,-0.10,5.209524


In [13]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
etf-11b-adjclose-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
etf-11b-high-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
etf-11b-low-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras

sent 28,829,795 bytes  received 73 bytes  19,219,912.00 bytes/sec
total size is 57,647,576  speedup is 2.00
